In [1]:
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from configs import g_key

In [3]:
#Load csv in part 1 to a dataframe

#clean city data filepath
path = "../output_data/clean_city_data.csv"

city_df = pd.read_csv(path)
city_df = city_df.dropna()
city_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint george,37.10,-113.58,73.400,20,1,9.171439,US,1593089596
1,busselton,-33.65,115.33,57.002,55,5,8.522727,AU,1593089596
2,pimentel,-6.84,-79.93,60.800,87,92,6.934503,PE,1593089596
3,rikitea,-23.12,-134.97,72.374,74,2,13.690050,PF,1593089596
4,faanui,-16.48,-151.75,78.566,88,6,16.844130,PF,1593089596


# Humidity Heatmap

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
#Store Lat and Lng into locatins
locations = city_df[["Lat", "Lng"]]

#Store humidity into humidity
humidity = city_df["Humidity"].astype(float)

In [6]:
#Create humidity Heatmap layer
fig = gmaps.figure()

humid_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                 max_intensity = 100, point_radius = 1)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [7]:
#Narrow down cities to fit weather conditions

#Filter cities with max temps between 60 and 90 degrees
comfortable_df = city_df.loc[(city_df["Max Temp"] > 60) & (city_df["Max Temp"] < 90), :]
#filter by humidity below 80 percent
comfortable_df = comfortable_df.loc[(comfortable_df["Humidity"] < 80), :]
#filter by wind speed below 10 mph
comfortable_df = comfortable_df.loc[(comfortable_df["Wind Speed"] < 10), :]
#filter by cloudiness below 50 %
comfortable_df = comfortable_df.loc[(comfortable_df["Cloudiness"] < 50), :]

#drop rows with na values
comfortable_df = comfortable_df.dropna()

In [8]:
#store into variable 'hotel_df'
hotel_df = comfortable_df
#adds column "Hotel_name"
hotel_df["Hotel Name"] = ""

#finding a hotel within 5000 meters of our most comfortable cities
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = row.Lat
    lng = row.Lng
    params = {
        "location": f'{lat}, {lng}',
        "radius": 5000,
        "key" : g_key,
        "type" : "lodging"
    }
    response = requests.get(base_url, params = params).json()
    #updates df with the hotel name
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    #displays message about the issue city and adds "NA" to hold the Hotel name's place
    except:
        print(f'Issues with finding a hotel at {row["City"]}')
        hotel_df.loc[index, "Hotel Name"] = "NA"
        
hotel_df.head()

Issues with finding a hotel at aripuana
Issues with finding a hotel at belaya gora
Issues with finding a hotel at verkhoyansk
Issues with finding a hotel at kedrovyy
Issues with finding a hotel at niquero
Issues with finding a hotel at chokurdakh
Issues with finding a hotel at kuloy
Issues with finding a hotel at aykhal
Issues with finding a hotel at sinegorye
Issues with finding a hotel at kamenka
Issues with finding a hotel at sangar
Issues with finding a hotel at praxedis guerrero
Issues with finding a hotel at uvinza
Issues with finding a hotel at dauriya
Issues with finding a hotel at dekar
Issues with finding a hotel at alekseyevsk
Issues with finding a hotel at rachitoasa


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint george,37.10,-113.58,73.400,20,1,9.171439,US,1593089596,Best Western Plus Abbey Inn
8,east london,-33.02,27.91,78.800,20,0,5.816034,ZA,1593089596,Tu Casa
9,port alfred,-33.59,26.89,84.002,22,0,7.001611,ZA,1593089596,The Halyards Hotel
16,paldiski,59.36,24.05,78.800,39,0,9.171439,EE,1593089598,Pakri Hostel
23,jamestown,42.10,-79.24,60.800,77,20,5.816034,US,1593089406,Hampton Inn & Suites Jamestown


In [9]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

#Add markers on top of the heatmap
hotel_layer = gmaps.symbol_layer(
    locations, fill_color = "blue",
    stroke_color = 'black', scale = 2,
    info_box_content = hotel_info)

fig.add_layer(hotel_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))